In [ ]:
import ipynb.fs.full.helper as h

from imp import reload
reload(h)

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
spec = model_spec.get('efficientdet_lite0') # maybe different lite1-4
#spec.model_dir="/logs/fit/test/"

In [ ]:
def read_train_val_test(dataset_dir, label_map):
    return tuple(map(lambda set_: object_detector.DataLoader.from_pascal_voc(images_dir=os.path.join(dataset_dir, set_), annotations_dir=os.path.join(dataset_dir, set_), label_map=label_map), ("train", "val", "test")))

In [ ]:
train_data, validation_data, test_data = read_train_val_test(h.datasetsTf, h.query_array)

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=16, train_whole_model=True, validation_data=validation_data, epochs=5)

In [ ]:
model.evaluate(test_data)

In [ ]:
model.export(export_dir=h.modelsVTf)

In [ ]:
model.evaluate_tflite(h.modelsVTf + '/model.tflite', validation_data)